<a href="https://colab.research.google.com/github/NvdSuni/Thesis-code-complete/blob/main/MRnet_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

from tensorflow import keras
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import sklearn
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib
from tensorflow.keras import layers
from sklearn import __version__ as sklearn_version

print(f"pandas: {pd.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"numpy: {np.__version__}")
print(f"tensorflow: {tf.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"scikit-learn: {sklearn_version}")


In [ ]:
pip install timm

In [ ]:
data_dir = '/content/drive/MyDrive/Master Thesis/MRNet-v1.0/train'

#Define the common dimensions
new_slices = 17
new_height = 256
new_width = 256

num_scans = 1130

#Process and save the sagittal plane
cropped_sagittal = np.zeros((num_scans, new_slices, new_height, new_width))
for i in range(num_scans):
    sagittal_filename = os.path.join(data_dir, 'sagittal', f'{i:04d}.npy')
    sagittal_scan = np.load(sagittal_filename)

    start_slice = max((sagittal_scan.shape[0] - new_slices) // 2, 0)

    #Extract middle slices
    sagittal_scan = sagittal_scan[start_slice:start_slice + new_slices, :new_height, :new_width]

    cropped_sagittal[i] = sagittal_scan

output_file_sagittal = '/content/drive/MyDrive/Master Thesis/MRNet-v1.0/cropped_sagittal.npy'
np.save(output_file_sagittal, cropped_sagittal)

In [ ]:
data_dir = '/content/drive/MyDrive/Master Thesis/MRNet-v1.0/test'

#Define the common dimensions
new_slices = 17
new_height = 256
new_width = 256

num_scans = 120

#Process and save the sagittal plane
cropped_sagittal = np.zeros((num_scans, new_slices, new_height, new_width))
for i in range(num_scans):
    sagittal_filename = os.path.join(data_dir, 'sagittal', f'{i:04d}.npy')
    sagittal_scan = np.load(sagittal_filename)

    start_slice = max((sagittal_scan.shape[0] - new_slices) // 2, 0)

    #Extract the middle slices
    sagittal_scan = sagittal_scan[start_slice:start_slice + new_slices, :new_height, :new_width]

    cropped_sagittal[i] = sagittal_scan

output_file_sagittal = '/content/drive/MyDrive/Master Thesis/MRNet-v1.0/cropped_sagittal_test.npy'
np.save(output_file_sagittal, cropped_sagittal)

In [ ]:
MRnet_dataset = os.chdir("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0")
data_train = pd.read_csv("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/train-complete.csv", encoding='utf-8')
data_sagittal = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/cropped_sagittal.npy")
X_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/cropped_sagittal_test.npy")

#Preprocessing

In [ ]:
data_train = data_train.drop(["ID"], axis = "columns")
data_train.head()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data_sagittal, data_train, test_size = 0.2, random_state = 42)

In [ ]:
healthy_samples = (y_train["abnormal"] == 0)

y_train["healthy"] = np.where(healthy_samples, 1, 0)

healthy_samples_val = (y_val["abnormal"] == 0)
y_val["healthy"] = np.where(healthy_samples_val, 1, 0)

In [ ]:
y_train["label"] = y_train.apply(lambda row: [row["abnormal"], row["acl"], row["meniscus"], row["healthy"]], axis=1)
y_val["label"] = y_val.apply(lambda row: [row["abnormal"], row["acl"], row["meniscus"], row["healthy"]], axis=1)

y_train = np.array(y_train["label"].tolist())
y_val = np.array(y_val["label"].tolist())

print(y_train)

In [ ]:
y_train_first_column = y_train[:, 0]
y_val_first_column = y_val[:, 0]

In [ ]:
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_MRNet.npy", y_train_first_column)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_MRNet.npy", y_val_first_column)

In [ ]:
#Normalisation
X_train_normalized = X_train / 255.0
X_val_normalized = X_val / 255.0
X_test_MRNet_normalised = X_test_MRNet / 255.0

In [ ]:
#Standardisation
mean = np.mean(X_train_normalized, axis=(0, 1, 2, 3))
std = np.std(X_train_normalized, axis=(0, 1, 2, 3))

#Standardize the data for training and validation sets
X_train_standardized = (X_train_normalized - mean) / std
X_val_standardized = (X_val_normalized - mean) / std
X_test_MRNet_normalised_standardised = (X_test_MRNet_normalised - mean) / std


In [ ]:
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/X_train_standardized_MRNet", X_train_standardized)
np.save("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/X_val_standardized_MRNet",X_val_standardized)

In [ ]:
X_train_standardized = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/X_train_standardized_MRNet.npy")
X_val_standardized = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/X_val_standardized_MRNet.npy")
y_train = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_train_MRNet.npy")
y_val = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_val_MRNet.npy")
X_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/MRNet-v1.0/X_test_MRNet.npy")
y_test_MRNet = np.load("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/y_test_MRNet.npy")

#FEATURE EXTRACTION


In [ ]:
model = keras.Sequential()

#Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(17, 256, 256, 1), name="COV1"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', name="COV2"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten(name="flatten"))

model.add(Dense(128, activation='relu', name="dense_layer_1"))
model.add(Dense(1, activation='sigmoid', name="output_layer"))  # Use 1 unit with sigmoid for binary classification

#Define a custom learning rate
custom_learning_rate = 0.00000001

optimizer = Adam(learning_rate=custom_learning_rate)

#Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_standardized, y_train, batch_size=8, epochs=20, validation_data=(X_val_standardized, y_val))


In [ ]:
#Extract features from the CNN model
cnn_feature_extractor_MRI = Model(inputs=model.input, outputs=model.get_layer('COV2').output)

cnn_feature_extractor_MRI.save('cnn_feature_extractor_model_MRI')



In [ ]:
cnn_feature_extractor_MRI = load_model('cnn_feature_extractor_model_MRI')

In [ ]:
batch_size = 8

cnn_features_X_train_MRNet = []

for i in range(0, len(X_train_standardized), batch_size):
    batch_data = X_train_standardized[i:i + batch_size]
    features = cnn_feature_extractor_MRI.predict(batch_data)
    cnn_features_X_train_MRNet.append(features)

#Concatenate the results
cnn_features_X_train_MRNet = np.concatenate(cnn_features_X_train_MRNet, axis=0)

np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_train_MRNet2.npy", cnn_features_X_train_MRNet)

In [ ]:
cnn_features_X_val_MRNet = []

for i in range(0, len(X_val_standardized), batch_size):
    batch_data = X_val_standardized[i:i + batch_size]
    features = cnn_feature_extractor_MRI.predict(batch_data)
    cnn_features_X_val_MRNet.append(features)

#Concatenate the results
cnn_features_X_val_MRNet = np.concatenate(cnn_features_X_val_MRNet, axis=0)

np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_val_MRNet2.npy", cnn_features_X_val_MRNet)

In [ ]:
cnn_features_X_test_MRNet = []

for i in range(0, len(X_test_MRNet_normalised_standardised), batch_size):
    batch_data = X_test_MRNet_normalised_standardised[i:i + batch_size]
    features = cnn_feature_extractor_MRI.predict(batch_data)
    cnn_features_X_test_MRNet.append(features)

#Concatenate the results
cnn_features_X_test_MRNet = np.concatenate(cnn_features_X_test_MRNet, axis=0)

np.save("/content/drive/My Drive/Tilburg University/Master Thesis/Combined data/cnn_features_X_test_MRNet.npy", cnn_features_X_test_MRNet)